In [1]:
import os
import numpy as np

import pandas as pd

In [2]:
data = pd.read_csv("../avatars_echOvid_train_X.csv")

data

,id,Nom du centre,age,Sexe,Lieu de provenance du patient,Echographiste,Tabagisme actif,BPCO,Asthme,Autre antecedent respiratoire,...,Zone anterieure droite haut,Zone anterieure droite bas,Zone anterieure gauche haut,Zone anterieure gauche bas,Zone posterieure droite haut,Zone posterieure droite bas,Zone posterieure gauche haut,Zone posterieure gauche bas,Oxygenotherapie,Outcome J28
0,1,Paris-APHP - Cochin,91,Masculin,Domicile,Forme pour l'epidemie,Non,Non,Non,Non,...,Stade 2,Stade 2,Stade 1,Stade 2,Stade 2,Stade 2,Stade 2,Stade 2,NaN,Death
1,109,Paris-APHP - Hotel Dieu,45,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 1,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Moderee,Back home
2,46,Melun - CH Marc Jacquet,76,Feminin,EHPAD,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 1,Stade 1,Stade 0,Stade 1,Stade 2,Stade 1,Stade 2,Assistance respiratoire,Hospitalization
3,247,Paris-APHP - Hotel Dieu,55,Feminin,Hopital,Experience d'echographie,Oui,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
4,195,Paris-APHP - Hotel Dieu,42,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,218,Paris-APHP - Cochin,34,Masculin,Domicile,Forme pour l'epidemie,Oui,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
258,199,Melun - CH Marc Jacquet,58,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 1,Stade 1,Stade 0,Stade 0,Stade 1,Stade 1,Stade 0,Stade 1,Assistance respiratoire,Hospitalization
259,117,Paris-APHP - Cochin,45,Feminin,Domicile,Expert,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
260,201,Paris-APHP - Cochin,46,Feminin,Domicile,Expert,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
